# 資料前處理(Label encoding、 One hot encoding)
這兩個編碼方式的目的是為了將類別 (categorical)或是文字(text)的資料轉換成數字，而讓程式能夠更好的去理解及運算。
> Label encoding : 把每個類別 mapping 到某個整數，不會增加新欄位

> One hot encoding : 為每個類別新增一個欄位，用 0/1 表示是否

![](images/Encoder.PNG)


## Encoding Categorical features (or label)
![](images/Encoding.PNG)


In [83]:
import pandas as pd
import numpy as np


In [84]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,blood,Y,Z
0,A,high,NaN
1,B,low,NaN
2,AB,high,-1196.0
3,O,mid,72.0
4,B,mid,83.0


# 方法一：sklearn - label encoder + onehot encoder
>onehot encoder要用2D array，若維度所以要用reshape(-1,1)<br>
>onehot encoder要數字，若資料文文字要先用label encoder轉數字

In [85]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
print(encoder)
encoder_Y = encoder.fit_transform(df['blood'])
print(encoder_Y) #A,B,AB,O 共四種血型>encoder [0 2 1 3 2] 這四個數字
df['blood'] = encoder_Y #再把轉換好的次序丟回去df
df

LabelEncoder()
[0 2 1 3 2]


,blood,Y,Z
0,0,high,NaN
1,2,low,NaN
2,1,high,-1196.0
3,3,mid,72.0
4,2,mid,83.0


In [86]:
##OnehotEncoder 
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder() #Attributes: categories_list of arrays #是nunpy 的arrays，所以要把下面的變成arrays
print(type(onehot))

print(df['blood'])
# print(type(df['blood'])) #<class 'pandas.core.series.Series'>
d = np.array(df['blood']) #<class 'numpy.ndarray'>
d.shape #(5,)
# print(type(d))
print('d.reshape(-1,1):')
print(d.reshape(-1,1))

onehot_df = onehot.fit_transform(d.reshape(-1,1)) 
onehot_df = onehot.fit_transform(d.reshape(-1,1)).toarray()
print(type(onehot_df))
onehot_df



<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
0    0
1    2
2    1
3    3
4    2
Name: blood, dtype: int64
d.reshape(-1,1):
[[0]
 [2]
 [1]
 [3]
 [2]]
<class 'numpy.ndarray'>


array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

## One hot encoding
One Hot encoding的編碼邏輯為將類別拆成多個行(column)，每個列中的數值由1、0替代，當某一列的資料存在的該行的類別則顯示1，反則顯示0。

然在指定column進行編碼的情形下，One hot encoding<b>無法直接對字串進行編碼，必須先透過Label encoding將字串以數字取代後再進行One hot encoding處理。</b>

> categorical_features = [0]: 表示欲在data上執行One hot encoding的index為0

> data_le: 為經過Label encoding編碼的資料(註:OneHotEncoder的輸入要為2-D array，而Label encoding為1-D array)


OneHotEncoder會轉出scipy.csr_matrix資料結構用.toarray()轉array
從結果可以知道，數字0的column 代表的是A、數字1的column 代表的是B，而數字2的column 代表的是AB。
除了轉換字串外，One hot encoding也可以轉換數字。在此處的data就不需要先經過Label encoding編碼

```python
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
  
data = np.array(columnTransformer.fit_transform(data), dtype = str) 
```

In [87]:
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder'
                     ,OneHotEncoder() 
                     ,[0])] #針對第0個欄位要做OneHotEncoder()
                     ,remainder='passthrough') #如果沒有被指定的欄位會被刪掉?)

data = np.array(columnTransformer.fit_transform(df) ,dtype = str) #type <class 'numpy.ndarray'> #本來的資料型態是array，就不能用下面的
# np.array(columnTransformer.fit_transform(df)  是array就不能用再用.toarray()
print(type(data)) 
data_le = pd.DataFrame(data)
data_le


<class 'numpy.ndarray'>


,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,high,nan
1,0.0,0.0,1.0,0.0,low,nan
2,0.0,1.0,0.0,0.0,high,-1196.0
3,0.0,0.0,0.0,1.0,mid,72.0
4,0.0,0.0,1.0,0.0,mid,83.0


# 方法二：Keras - label encoder + to_categorical
>to_categorical要數字，若資料文文字要先用label encoder轉數字

In [88]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});

# label encoder 
encoder = LabelEncoder()
encoder_Y = encoder.fit_transform(df['blood'])
print(encoder_Y)
df['blood'] = encoder_Y
df

# convert integers to one hot encoding
keras_onehot = np_utils.to_categorical(encoder_Y)
print(type(keras_onehot))
keras_onehot



[0 2 1 3 2]
<class 'numpy.ndarray'>


array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]], dtype=float32)

## 方法三：pd.get_dummies方法
![](images/Encoding_pd.PNG)
pd.get_dummies(df)
>get_dummies可以直接轉字串，反而無法轉換數字<br>
>get_dummies沒指定columns，會全部轉換

In [89]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]})
df1 = pd.get_dummies(df)
print(df1)
df2 = pd.get_dummies(df.blood)
print(df2)

        Z  blood_A  blood_AB  blood_B  blood_O  Y_high  Y_low  Y_mid
0     NaN        1         0        0        0       1      0      0
1     NaN        0         0        1        0       0      1      0
2 -1196.0        0         1        0        0       1      0      0
3    72.0        0         0        0        1       0      0      1
4    83.0        0         0        1        0       0      0      1
   A  AB  B  O
0  1   0  0  0
1  0   0  1  0
2  0   1  0  0
3  0   0  0  1
4  0   0  1  0


## 練習一：sklearn - label encoder + onehot encoder
下面的資料可以看到country那欄皆為字串， 大部分的模型都是基於數學運算，字串無法套入數學模型進行運算，<br>
在此先對其進行Label encoding編碼，我們從 sklearn library中導入 LabelEncoder class，對第一行資料進行fit及transform並取代之。

In [90]:
import numpy as np
import pandas as pd
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


In [91]:

from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder() 

d = np.array(data['Country']) #<class 'numpy.ndarray'>
print(d.reshape(-1,1))
onehot_df = onehot.fit_transform(d.reshape(-1,1)) 
onehot_df = onehot.fit_transform(d.reshape(-1,1)).toarray()
onehot_df

[['Taiwan']
 ['Australia']
 ['Ireland']
 ['Australia']
 ['Ireland']
 ['Taiwan']]


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [92]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 


columnTransformer = ColumnTransformer([('encoder'
                     ,OneHotEncoder() 
                     ,[0])] #針對第0個欄位要做OneHotEncoder()
                     ,remainder='passthrough') #如果沒有被指定的欄位會被刪掉?)

data = np.array(columnTransformer.fit_transform(data) ,dtype = str) 
print(type(data)) 
data_le = pd.DataFrame(data)
data_le

<class 'numpy.ndarray'>


,0,1,2,3,4
0,0.0,0.0,1.0,25.0,20000.0
1,1.0,0.0,0.0,30.0,32000.0
2,0.0,1.0,0.0,45.0,59000.0
3,1.0,0.0,0.0,35.0,60000.0
4,0.0,1.0,0.0,22.0,43000.0
5,0.0,0.0,1.0,36.0,52000.0


## 練習二：Keras - label encoder + to_categorical

In [93]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


In [94]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# label encoder 
encoder = LabelEncoder()
encoder_Y = encoder.fit_transform(data['Country'])

data['Country'] = encoder_Y
data

# convert integers to one hot encoding
keras_onehot = np_utils.to_categorical(encoder_Y)
print(type(keras_onehot))
keras_onehot

<class 'numpy.ndarray'>


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

## 練習三：Pandas.get_dummies
>　get_dummies : 僅能將字串轉換為One hot encoding表示形式， 沒指定columns會全部轉換。

In [95]:
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


In [96]:
df1 = pd.get_dummies(data)
print(df1)
df2 = pd.get_dummies(data.Country)
print(df2)

   Age  Salary  Country_Australia  Country_Ireland  Country_Taiwan
0   25   20000                  0                0               1
1   30   32000                  1                0               0
2   45   59000                  0                1               0
3   35   60000                  1                0               0
4   22   43000                  0                1               0
5   36   52000                  0                0               1
   Australia  Ireland  Taiwan
0          0        0       1
1          1        0       0
2          0        1       0
3          1        0       0
4          0        1       0
5          0        0       1
